# Clustering Neighbourhoods of Toronto

In this notebook, the geographical data will be used along with the foursquare API to find geographical information about each neighbourhood.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # convert address to lat and long

import requests # handle url requests

from pandas.io.json import json_normalize # transform JSON into DataFrame

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # k-mean clustering

import folium # map rendering

Load the neighbourhood data from the last part

In [3]:
nbors = pd.read_csv('Neighbourhoods_of_Toronto_Lat_Long.csv', index_col=0)
nbors.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [6]:
print('The dataframe has {} boroughs and {} Neighbourhoods'.format(len(nbors['Borough'].unique()), len(nbors['Neighbourhood'].unique())))

The dataframe has 10 boroughs and 99 Neighbourhoods


In [8]:
nbors.shape

(103, 5)

There appears to be a mismatch, this suggests that some neighbourhoods are duplicated.

In [15]:
n_counts=nbors['Neighbourhood'].value_counts()
n_counts[n_counts > 1] # find neighbourhoods with more than one entry

Downsview    4
Don Mills    2
Name: Neighbourhood, dtype: int64

In [16]:
for i in n_counts[n_counts >1].index:
    print(nbors[nbors['Neighbourhood'] == i])

   Postal Code     Borough Neighbourhood   Latitude  Longitude
30         M3K  North York     Downsview  43.737473 -79.464763
31         M3L  North York     Downsview  43.739015 -79.506944
32         M3M  North York     Downsview  43.728496 -79.495697
33         M3N  North York     Downsview  43.761631 -79.520999
   Postal Code     Borough Neighbourhood   Latitude  Longitude
26         M3B  North York     Don Mills  43.745906 -79.352188
27         M3C  North York     Don Mills  43.725900 -79.340923


Although there are duplicate neighbourhoods, they have different postcodes and different longitude and latitudes. Let's just leave them in for now.

#### Now set up geopy to find geographical coordinates of Toronto

In [20]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent='toronto_explorer') # define a user agent for geopy
location = geolocator.geocode(address) # find the location of our address
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


#### Now create a map of Toronto with Folium

In [1]:
# create the map
map_toronto = folium.Map(location=[latitude+0.075, longitude], zoom_start=11) #shift latitude a bit for centering

#add markers of neighbourhoods
for lat, lng, borough, neighbourhood in zip(nbors['Latitude'], nbors['Longitude'], nbors['Borough'], nbors['Neighbourhood']):
    label='{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        foll_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

NameError: name 'folium' is not defined

#### Now define foursquare credential and version

In [46]:
with open('foursquare_credentials.txt') as f:
    l = f.readline()
    cred = json.loads(l)

client_id = cred['CLIENT_ID']
client_secret = cred['CLIENT_SECRET']
access_token = cred['ACCESS_TOKEN']
version = cred['VERSION']
limit = str(cred['LIMIT'])

#### Now get venues within 500m of each neighbourhood.
Without knowing the spacing of each neighbourhood it may be better to use a larger or smaller distance. We will use the explore query of foursqaure for this

In [48]:
#to try it, let's get lat and long for the first item in the dataframe
nbors_1 = nbors.iloc[1]
lat = nbors_1['Latitude']
lng = nbors_1['Longitude']
print('Latitude and Longitude is {}, {} for the neighbourhood {} in the borough {}.'.format(lat, lng, nbors_1['Neighbourhood'], nbors_1['Borough']))

Latitude and Longitude is 43.78453510000001, -79.16049709999999 for the neighbourhood Rouge Hill, Port Union, Highland Creek in the borough Scarborough.


In [50]:
radius = '500'
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{},&v={}&radius={}&limit={}'.format(client_id, client_secret, access_token, str(lat), str(lng), version, radius, limit)

In [52]:
#Now get the results from the query
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fc98acb7c1f474e47294159'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.78903510450001,
    'lng': -79.15427558741125},
   'sw': {'lat': 43.780035095500004, 'lng': -79.16671861258872}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '555c7b97498e157737abdcd6',
       'name': 'RIGHT WAY TO GOLF',
       'location': {'address': '13-229 Meadowvale Road',
        'lat': 43.785176919005714,
        'lng': -79.16110754013062,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.785176919005714,
          'lng': -79.161

##### Explore the results a bit to see what type of venues are nearby

In [58]:
# using the get categrory function from the workshops
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [62]:
venues = results['response']['groups'][0]['items']
venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

#just select columns of interest
venues = venues.loc[:, filtered_columns]

#get category
venues['venue.categories'] = venues.apply(get_category_type, axis=1)

#neaten columns

venues.columns = [col.split('.')[-1] for col in venues.columns]

venues.head()

,name,categories,lat,lng
0,RIGHT WAY TO GOLF,Golf Course,43.785177,-79.161108
1,Chris Effects Painting,Construction & Landscaping,43.784343,-79.163742
2,Great Shine Window Cleaning,Home Service,43.783145,-79.157431
3,SEBS Engineering Inc. (Sustainable Energy and ...,Construction & Landscaping,43.782371,-79.156820
4,Affordable Toronto Movers,Moving Target,43.787919,-79.162977


In [63]:
print('{} venues were returned by foursquare'.format(venues.shape[0]))

5 venues were returned by foursquare


### Repeat this process or all neighbourhoods in Toronto


In [69]:
#use the same function from the workshop
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
        print(url)    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now to run the above function through all the neighbourhoods in the nbors dataframe.

In [70]:
toronto_venues = getNearbyVenues(nbors['Neighbourhood'].values, nbors['Latitude'].values, nbors['Longitude'].values)

Malvern, Rouge
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100
Rouge Hill, Port Union, Highland Creek
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.78453510000001,-79.16049709999999&radius=500&limit=100
Guildwood, Morningside, West Hill
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100
Woburn
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.7709921,-79.2

Downsview
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.7284964,-79.49569740000001&radius=500&limit=100
Downsview
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.7616313,-79.52099940000001&radius=500&limit=100
Victoria Village
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.72588229999999,-79.31557159999998&radius=500&limit=100
Parkview Hill, Woodbine Gardens
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.7063972,-79.30993699999998&radius=500&li

The Annex, North Midtown, Yorkville
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.6727097,-79.40567840000001&radius=500&limit=100
University of Toronto, Harbord
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.6626956,-79.40004929999998&radius=500&limit=100
Kensington Market, Chinatown, Grange Park
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.6532057,-79.40004929999998&radius=500&limit=100
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.6435152,-79.57720079999999&radius=500&limit=100
Humber Summit
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.7563033,-79.56596329999998&radius=500&limit=100
Humberlea, Emery
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.7247659,-79.53224240000002&radius=500&limit=100
Weston
https://api.foursquare.com/v2/venues/explore?&client_id=ABHFVIJDJT1DULBEFNC55BZZIGN52DM4V1NLRRDWDZ1TUJXM&client_secret=CR322OHATSRPLII1QZLWYWLTHBI4NAPGUCLU4AOUT51ELCVD&v=20180605&ll=43.706876,-79.51818

In [71]:
print(toronto_venues.shape)
toronto_venues.head()

(2133, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Check number of venues for each neighbourhood

In [72]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16


Number on unique categories

In [74]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 unique categories.


### Analyse each neighbourhood

In [75]:
# one hot encoding
toronto_one_hot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighbourhood column and move to the front
toronto_one_hot['Neighbourhood'] = toronto_venues['Neighbourhood']
fixedcolumn = [toronto_one_hot.columns[-1]] + list(toronto_one_hot.columns[:-1])
toronto_one_hot = toronto_one_hot[fixedcolumn]

toronto_one_hot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

New dataframe size:

In [76]:
toronto_one_hot.shape

(2133, 274)

Group rows by neighbourhood and take the mean of the freuqency of occurence

In [103]:
toronto_grouped = toronto_one_hot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.

In [83]:
toronto_grouped.shape

(96, 274)

Let's examine the top 5 most common venues

In [85]:
top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print('----'+hood+'----')
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:] # remove neighbourhood row
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2}) #round frequency to 2 places
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store   0.2
1             Breakfast Spot   0.2
2                     Lounge   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1             Gym  0.14
2  Sandwich Place  0.14
3             Pub  0.14
4     Coffee Shop  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.10
1        Coffee Shop  0.10
2  Mobile Phone Shop  0.05
3              Diner  0.05
4        Gas Station  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3                 Café  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1      Sandwich Place  0.09
2  Italian Restaurant  0.09
3    Greek Restaura

                 venue  freq
0          Coffee Shop  0.08
1            Gastropub  0.05
2              Brewery  0.05
3  American Restaurant  0.05
4                 Café  0.05


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
                 venue  freq
0          Coffee Shop  0.14
1          Pizza Place  0.07
2                  Pub  0.07
3         Liquor Store  0.07
4  Fried Chicken Joint  0.07


----The Annex, North Midtown, Yorkville----
            venue  freq
0  Sandwich Place  0.16
1            Café  0.16
2     Coffee Shop  0.11
3  History Museum  0.05
4       BBQ Joint  0.05


----The Beaches----
               venue  freq
0  Health Food Store  0.25
1              Trail  0.25
2       Neighborhood  0.25
3                Pub  0.25
4     Medical Center  0.00


----The Danforth West, Riverdale----
                venue  freq
0    Greek Restaurant  0.16
1         Coffee Shop  0.09
2  Italian Restaurant  0.07
3      Ice Cream Shop  0.05
4          Restaurant  

### Put all that into a dataframe

In [86]:
#using the same function from the workshop
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

#create columns
columns = ['Neighbourhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#create new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], top_venues)

venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Pharmacy,Gym,Sandwich Place,Coffee Shop,Pub,Dog Run,Dim Sum Restaurant,Diner,Discount Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Chinese Restaurant,Bridal Shop,Sandwich Place,Diner,Restaurant,Middle Eastern Restaurant,Supermarket
3,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Thai Restaurant,Locksmith,Liquor Store,Comfort Food Restaurant,Juice Bar,Butcher


It should be noted that some searched returned less than 10 results

### Cluster the neighbourhoods

Run k-means clustering on the neighbourhoods. To begin with try 5 clusters

In [89]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

#run clustering
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(toronto_grouped_clustering)

#check labels
kmeans.labels_[:30]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1])

Appears to be a lot of cluster 1

Create a new dataframe that includes the cluster as well as top 10 venues for each neighbourhood

In [90]:
#add cluster labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = nbors

#merge with grouped data
toronto_merged = toronto_merged.join(venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1.0,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Health & Beauty Service
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Breakfast Spot,Restaurant,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Mexican Restaurant,Yoga Studio,Doner Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Gas Station,Fried Chicken Joint,Bakery,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Electronics Store,Eastern European Restaurant


Visualise the clusters on the map. First some neighbourhoods did not get a cluster, presumably from not returning any information from foursquare. e.g. 'Upper Rouge' neighbourhood

In [110]:
toronto_merged.dropna(axis=0, how='any', inplace = True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int32')

In [112]:
#create map
toronto_merged['Cluster Labels'].astype('int32')
map_clusters = folium.Map(location=[latitude+0.075, longitude], zoom_start=11)

#set colour scheme for clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colour_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colour_array]

#add markers
markers_colours =[]
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi)+' Cluster '+str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popul=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

### Examining the clusters

#### Cluster 1

In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,0,Park,Convenience Store,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
25,North York,0,Park,Food & Drink Shop,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
50,Downtown Toronto,0,Park,Playground,Trail,Yoga Studio,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
64,Central Toronto,0,Trail,Park,Sushi Restaurant,Jewelry Store,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
74,York,0,Park,Women's Store,Pool,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
98,York,0,Park,Yoga Studio,Eastern European Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


#### Cluster 2

In [117]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Fast Food Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Health & Beauty Service
1,Scarborough,1,Construction & Landscaping,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,Scarborough,1,Breakfast Spot,Restaurant,Rental Car Location,Electronics Store,Medical Center,Intersection,Bank,Mexican Restaurant,Yoga Studio,Doner Restaurant
3,Scarborough,1,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,Scarborough,1,Gas Station,Fried Chicken Joint,Bakery,Bank,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Electronics Store,Eastern European Restaurant
5,Scarborough,1,Smoke Shop,Playground,Jewelry Store,Yoga Studio,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
6,Scarborough,1,Department Store,Hobby Shop,Coffee Shop,Train Station,Event Space,Ethiopian Restaurant,Escape Room,Falafel Restaurant,Electronics Store,Dim Sum Restaurant
8,Scarborough,1,Motel,American Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
9,Scarborough,1,College Stadium,Café,Skating Rink,General Entertainment,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
10,Scarborough,1,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


#### Cluster 3

In [118]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,2,Print Shop,Yoga Studio,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dim Sum Restaurant


#### Cluster 4

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,3,Bus Line,Bakery,Intersection,Soccer Field,Park,Ice Cream Shop,Metro Station,Bus Station,Doner Restaurant,Donut Shop
14,Scarborough,3,Intersection,Park,Playground,Bakery,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
40,East York,3,Intersection,Park,Convenience Store,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,3,Park,Swim School,Bus Line,Yoga Studio,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
72,North York,3,Park,Pizza Place,Japanese Restaurant,Bakery,Pub,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
79,North York,3,Park,Bakery,Construction & Landscaping,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
100,Etobicoke,3,Park,Pizza Place,Sandwich Place,Bus Line,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore


#### Cluster 5

In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,4,Construction & Landscaping,Baseball Field,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
97,North York,4,Baseball Field,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Field


#### Cluster 1
Mostly fitness facilities and some general stores
#### Cluster 2
The biggest cluster, predominately restaurants and coffee shops with a few general stores
#### Cluster 3
Cluster 3 only has one entry, the third most common venue and onwards appears to be in alphabetical order, suggesting there is only a print shop and yoga studio
#### Cluster 4
Cluster 4 is focussed on parks as well as general anmenities
#### Cluster 5
Cluster 5 has more fitness or sport type venues, with restaurants being less common

It should be noted that as a lot of neighbourhoods did not have more that 3-4 items on foursquare within a radius of 500m, some of the common venues are very similar. This may have affected the clustering and future examination may find it beneficial to either increase the radius or remove neighbourhoods not in cluster 2. There is also a large concentration of neighbourhoods in central toronto, so perhaps increasing the radius is not an option.